# <a href="https://www.pythontutorial.net/advanced-python/python-context-managers/" style="color:Tomato">Python Context Managers</a>

Bài này ta học về Python context manager và cách sử dụng nó hiệu quả.

### Tables of Contents
* [Introduction to Python context managers](#1)
* [Python `with` statement](#2)
* [Python context manager protocol](#3)
    - [The `__enter__()` method](#3.1)
    - [The `__exit__()` method](#3.2)
* [Python context manager applications](#4)
    - [Open – Close](#4.1)
    - [Lock – release](#4.2)
    - [Start – stop](#4.3)
    - [Change – reset](#4.4)
* [Using Python context manager to implement the start and stop pattern](#4)
* [Summary](#sum)

## <a class="anchor" id="1">Introduction to Python context managers</a>

<span style="color:DarkOrange">Một **context manager** là một object định nghĩa một **runtime context** thực thi bên trong lệnh `with`.</span>

Hãy bắt đầu với một ví dụ đơn giản để hiểu hơn về context manager.

Giả sử ta có một file là `data.txt` có chứa một số nguyên là `100`.

Chương trình sau đọc file `data.txt`, convert nội dung file thành số nguyên, và hiển thị kết quả ra standard output:

```python
f = open('data.txt')
data = f.readlines()

# convert the number to integer and display it
print(int(data[0]))

f.close()
```

Code này nhìn rất đơn giản và dễ hiểu. Tuy nhiên, file `data.txt` có thể chứa những data mà không thể được convert sang dạng số. Trong trường hợp này, code sẽ raise một cái exception nào đấy.

Ví dụ, nếu file `data.txt` chứa một string `'100'` thay vì số `100`, ta sẽ được lỗi sau:

```
ValueError: invalid literal for int() with base 10: "'100'"
```

Nếu nó xảy ra, Python có thể sẽ không đóng file đúng cách.

Để khắc phục, ta có thể sử dụng cấu trúc `try...except...finally`:

```python
try:
    f = open('data.txt')
    data = f.readlines()
    # convert the number to integer and display it
    print(int(data[0]))
except ValueError as error:
    print(error)
finally:
    f.close()
```

Vì phần code trong `finally` block luôn được thực thi, chương trình sẽ đảm bảo file luôn được đóng đúng cách.

Chương trình hoạt động như mong đợi, nhưng nhìn nó có vẻ dài dòng và rườm rà.

Vì thế, Python cung cấp một cách để ta có thể đóng file đúng cách sau khi ta hoàn thành xử lý file. Đây là lúc mà **context manager** phát huy tác dụng.

Ví dụ sau sử dụng context manager để xử lý file `data.txt`:

```python
with open('data.txt') as f:
    data = f.readlines()
    print(int(data[0])    
```

Ở ví dụ này, ta sử dụng hàm `open()` sau câu lệnh `with`. Sau khi thực hiện block `with`, Python sẽ tự động đóng file.

## <a class="anchor" id="2">Python `with` statement</a>

Dưới đây là cú pháp của lệnh `with`:

```python
with context as ctx:
    # use the the object 

# context is cleaned up
```

How it works.

- Khi Python gặp lệnh `with`, nó sẽ tạo ra một context mới. Context này có thể optionall return một object.
- <span style="color:DarkOrange">Sau `with` block, Python tự động cleans up context đó.</span>
- <span style="color:DarkOrange">Scope của biến `ctx` chính là scope của lệnh `with`. Ta có thể access `ctx` ở bên trong và sau lệnh `with`.</span>

Ví dụ sau minh hoạ cách access biến `f` sau lệnh `with`:

```python
with open('data.txt') as f:
    data = f.readlines()
    print(int(data[0]))


print(f.closed)  # True
```

## <a class="anchor" id="3">Python context manager protocol</a>

<span style="color:DarkOrange">Python context manager hoạt động dựa vào **context manager protocol**.</span>

Context manager protocol có các phương thức sau:

- `__enter__()`: setup context và có thể return object.
- `__exit__()`: clean up the object

Nếu ta muốn một class support context manager protocol, ta có thể implement hai phương thức trên.

Giả sử ta có `ContextManager` là một class có support context manager protocol.

Ví dụ sau minh hoạ cách sử dụng class `ContextManager`:

```python
with ContextManager() as ctx:
    # do something
# done with the context
```

<span style="color:DarkOrange">Khi ta sử dụng class `ContextManager` với câu lệnh `with`, Python sẽ ngầm tạo một instance của class `ContextManager` và tự động gọi hàm `__enter__()` của instance đó.</span>

Hàm `__enter__()` có thể có hoặc không return một object. Nếu có, Python sẽ gán object được return vào biến `ctx`.

Chú ý là <span style="color:DarkOrange">biến `ctx` tham chiếu tới object được return bởi hàm `__enter__()`</span>, chứ không phải là instance của class `ContextManager`.

<span style="color:DarkOrange">Nếu một exception xảy ra bên trong block của lệnh `with`, Python sẽ gọi hàm `__exit__()` của instance object.</span>

![](https://www.pythontutorial.net/wp-content/uploads/2020/11/Python-Context-Manager.png)

Về mặt công dụng, <span style="color:DarkOrange">lệnh `with` tương đương với lệnh `try...finally`</span>:

```python
instance = ContextManager()
ctx = instance.__enter__()

try:
    # do something with the txt
finally:
    # done with the context
    instance.__exit__()
```

### <a class="anchor" id="3.1">The `__enter__()` method</a>

Bên trong hàm `__enter__()`, ta có thể chứa các bước cần thiết để setup context. Và có thể có hoặc không return một object từ hàm `__enter__()`.

### <a class="anchor" id="3.2">The `__exit__()` method</a>

Python luôn thực thi hàm `__exit__()` kể cả khi xảy ra exception ở bên trong block `with`.

Hàm `__exit__()` nhận vào 3 tham số: exception type, exception value, traceback object. Nếu không có exception, cả 3 tham số này đều là `None`.

```python
def __exit__(self, ex_type, ex_value, ex_traceback):
    ...
```

Hàm `__exit__()` return một giá trị boolean: `True` hoặc `False`.
- Nếu nó return `True`, nghĩa là exception đã được xử lý, ta không phải làm gì thêm.
- Nếu nó return `False`, nghĩa là exception chưa được xử lý, ta sẽ phải xử lý exception đó hoặc Python sẽ thông báo nó lên màn hình.

## <a class="anchor" id="4">Python context manager applications</a>

Như ta thấy ở ví dụ trước, cách sử dụng chung của context manager là để mở hoặc đóng file một cách tự động. Tuy nhiên, ta có thể sử dụng context manager trong nhiều trường hợp khác nữa.

### <a class="anchor" id="4.1">Open – Close</a>

Nếu ta muốn open hoặc close một tài nguyên một cách tự động, có thể dùng context manager.

Ví dụ, khi ta muốn mở một socket và đóng nó, sử dụng context manager.

### <a class="anchor" id="4.2">Lock – release</a>

Context manager có thể giúp chúng ta quản lý các locks một cách hiệu quả. Ta có thể accquire một lock và release nó một cách tự động.

> "lock" ở đây hiểu là cơ chế đồng bộ hoá, kiểm soát quyền truy cập đến một tài nguyên của các threads hoặc processes. Trong trường hợp này ta có thể ví dụ là hàm `__start__()` sẽ khoá một tài nguyên nào đấy, không cho các threads khác can thiệp vào và hàm `__exit__()` là để release nó.

### <a class="anchor" id="4.3">Start – stop</a>

Context managers cũng giúp cho ta làm việc với những thứ mà có start và stop.

Ví dụ ta có thể dùng context manager để start và stop một cái timer một cách tự động.

### <a class="anchor" id="4.4">Change – reset</a>

Ví dụ ta có một chương trình và nó cần connect tới nhiều data sources, nó cũng có một connection mặc định.

Để connect với một data source khác:

- Đầu tiên, sử dụng context manager để thay đổi connection mặc định.
- Tiếp theo, làm việc với connection mới đó
- Cuối cùng, reset nó về connection mặc định ban đầu sau khi đã làm việc xong với connection mới.

## <a class="anchor" id="5">Using Python context manager to implement the start and stop pattern</a>

Ví dụ sau định nghĩa một class `Timer` có support context manager protocol:

In [1]:
from time import perf_counter


class Timer:
    def __init__(self):
        self.elapsed = 0

    def __enter__(self):
        self.start = perf_counter()
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False

How it works.

- Đầu tiên import `perf_counter` của module `time`
- Sau đó, start timer ở trong hàm `__enter__()`
- Cuối cùng, stop timer ở trong hàm `__exit__()` và lưu lại thời gian đã chạy.

Bây giờ, ta có thể sử dụng class `Timer` để đo thời gian chạy cần thiết để tính được số fibonacci thứ 1000, trong 1000000 lần:

In [ ]:
def fibonacci(n):
    f1 = 1
    f2 = 1
    for i in range(n-1):
        f1, f2 = f2, f1 + f2

    return f1


with Timer() as timer:
    for _ in range(1, 1000000):
        fibonacci(1000)

print(timer.elapsed)

## <a class="anchor" id="sum" style="color:Violet"> Tổng kết </a>

- Sử dụng Python context manager để định nghĩa runtime contexts khi muốn thực thi nó với lệnh `with`.
- Implement hàm `__enter___()` và hàm `__exit__()` để support context manager protocol.